# report

1.EDA

2.预处理

暂时先做最简单的， NA->-1, Dummy 所有cata且个数<10的， std没做

3.BaseModel

Boosting Trees

RF

GridSearch

Fine Tuned Accuracy

    XGB
        param = {'eta': 0.02, 
                'max_depth': 4, 
                'subsample': 0.9, 
                'colsample_bytree': 0.9, 
                'objective': 'binary:logistic', 
                'eval_metric': 'auc', 
                'seed': 99, 
                'silent': True
               }  
    cv_accuracy = 0.2835
    test = 0.280

    LGBM1
        param = {
            'learning_rate' : 0.02,
            'n_estimators' : 650,
            'max_bin' : 10,
            'subsample' : 0.8,
            'subsample_freq' : 10,
            'colsample_bytree' : 0.8,
            'min_child_samples' : 500,
            'seed' : 99,
            }
    cv_accuracy = 0.2844
    test = 0.281

    LGBM2
        param = {
                'learning_rate' : 0.02,
                'n_estimators' : 1090,
                'subsample' : 0.7,
                'subsample_freq' : 2,
                'num_leaves' : 16,
                'seed' : 99,
                }
    cv_accuracy = 0.2833               
    test = 0.281

    RF
        param = {'n_estimators': 150,
                    'criterion': 'gini',
                    'max_features': 'log2',
                    'max_depth': 9,
                    'min_samples_split': 70,
                    'min_samples_leaf': 30,
                    'n_jobs' : -1,
                    'max_leaf_nodes': None,
                    'bootstrap': True,
                    'oob_score': True,
                    'random_state': 99,
                    'verbose': 0,
                    'warm_start': True,
                    'class_weight': {0: 0.0364, 1: 0.9635},
                    }
    cv_accuracy = 0.2589                 

4.Stacking

    stacking@test
        all_50%      : 0.272
        all_weighted : 0.273
        3_weighted   : 0.281
        3_50%        : 0.282

5.预处理测略

    不给dummy设置上限？变成227，看看看结果
    ->validation
        0.2840
        0.2842
        0.2838
    ->test@stacking
        0.283

    +missing&reg*car
        0.2844
        0.2851
        0.2839
    ->test@stacking
        0.282

    dimension reduction
    忽悠一下shadow
        0.2856
        0.2871
        0.2854
    ->test@stacking
        0.285

    NA的列都删了？(X)
    ->validation
        xgboost : 0.2529
        lgbm1   : 0.2502
        lgbm2   : 0.2510

std

    @xgboost
    None    0.2856     10:34 -> 12:01
    minmax  0.2855     13:23 -> 14:50 <-
    meanstd 0.2854     16:17 -> 17:48

NA

    bin/cata -> -1
    else ->mean
    @xg                0.2866
    @lgbm1             0.2881
    @lgbm2             0.2868
    ->test@stacking
        0.0285

    这里用EM既费时间假设不对又结果不好，只有binary比较好用GMM

Empirical Bayesian Encoding

https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf

    'ps_car_11_cat' throw origin
         @xgb     0.2881
         @lgbm1   0.2882
         @lgbm2   0.2880
         ->test@stacking
                  0.285
     >=10           throw origin  <-
        @xgb      0.2884
        @lgbm1    0.2892
        @lgbm2    0.2883
        ->test@stacking
                  0.285
     >=10           keep origin
        @xgb      
        @lgbm1    0.2876
        @lgbm2    0.2868
     >=3            throw origin
        @xgb      0.2843
                  0.2858
        @lgbm1    0.2850
        @lgbm2    0.2849       
     >=13
         0.2882
         0.2895[
         0.2892
         0.2881

类别没有被Dummy的进行分析

In [1]:
from script_preprocessing import *

In [2]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division
from __future__ import unicode_literals

import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold

from modeling import dataset
from preprocessing import Preprocessing as PP


In [3]:
# read csv as dataframe
filename = "./cv/cv_"
print("loading data to memory...")
train = pd.read_csv("./original/train.csv")
test = pd.read_csv("./original/test.csv")
# drop unrelated items
train = dataset(train.drop("id", 1).drop("target", 1),
                train.target.to_frame('target'))
test = dataset(test.drop("id", 1), test.id.to_frame('id'))
# -1 are actually NaN
train.X = train.X.replace(-1, np.nan)
test.X = test .X.replace(-1, np.nan)

loading data to memory...


In [4]:
train.X['ps_calc_01_log'] = np.log(train.X['ps_calc_01'])

In [5]:
train.X.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,ps_calc_01_log
0,2,2.0,5,1.0,0.0,0,1,0,0,0,...,1,5,8,0,1,1,0,0,1,-0.510826
1,1,1.0,7,0.0,0.0,0,0,1,0,0,...,1,1,9,0,1,1,0,1,0,-1.203973
2,5,4.0,9,1.0,0.0,0,0,1,0,0,...,2,7,7,0,1,1,0,1,0,-0.693147
3,0,1.0,2,0.0,0.0,1,0,0,0,0,...,2,4,9,0,0,0,0,0,0,-0.510826
4,0,2.0,0,1.0,0.0,1,0,0,0,0,...,1,1,3,0,0,0,1,1,0,-0.916291
